In [1]:
from numpy import *
import numpy as np
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt
from numpy import linalg as la
import pdb
%matplotlib inline

In [2]:
rr = pd.read_json('renttherunway_final_data.json',lines=True)
print(shape(rr))
rr.head(5)

category_list = ['gown','jacket']

rr = rr[rr['category'].isin(category_list)]

(192544, 15)


In [3]:
rr.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
1,fit,273551,34b,153475,132lbs,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36.0,"June 18, 2013"
4,fit,151944,34b,616682,145lbs,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,"5' 9""",12,27.0,"September 26, 2016"
8,fit,166228,36d,1729232,NaN,10.0,formal affair,I was nervous of it looking cheap when it arri...,full bust,Great for black tie event!,gown,"5' 6""",21,27.0,"June 27, 2016"
9,fit,154309,32b,1729232,114lbs,10.0,formal affair,The dress was very flattering and fit perfectl...,petite,This dress was everything! It was perfect for ...,gown,"5' 3""",1,33.0,"October 17, 2016"
14,fit,721308,34b,123793,118lbs,10.0,formal affair,"Fit great, super flattering",athletic,Stunning gown. Wore this for heart ball and re...,gown,"5' 5""",2,32.0,"May 29, 2014"


In [4]:
#reorder id's
item_id = rr['item_id']
user_id = rr['user_id']
rating = rr['rating']

rr.drop('item_id', axis = 1,inplace = True)
rr.insert(0,'item_id',item_id)
rr.drop('user_id', axis = 1, inplace  = True)
rr.insert(1,'user_id',user_id)
rr.drop('rating', axis = 1,inplace = True)
rr.insert(2,'rating',rating)

print(shape(rr))


(46785, 15)


In [7]:
rr_Sorted = rr.sort_values(['user_id'])
n_rr = rr_Sorted.drop_duplicates(['user_id'])
shape(n_rr)

n_rr_Sorted = n_rr.sort_values(['item_id'])
n_rr = n_rr_Sorted.drop_duplicates(['item_id'])
shape(n_rr)

n_rr_rating = n_rr.iloc[:,0:3]

n_rr_rating.item_id = n_rr_rating.item_id.astype(object)
n_rr_rating.user_id = n_rr_rating.user_id.astype(object)

##### User x Item Matrix

In [8]:
# user x item
R = n_rr_rating.pivot_table(index = ['user_id'],columns = ['item_id'],values = "rating")
R1 = R.replace(np.nan,0)
print(shape(R1))
print(R1.head())
R1 = np.array(R1)

(849, 849)
item_id  123373   123793   125465   125564   127081   127865   128959   \
user_id                                                                  
35           0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2132         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2968         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4170         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
5790         0.0      0.0      0.0      0.0      0.0      0.0      0.0   

item_id  131117   131533   131698   ...  2935830  2935879  2941989  2945782  \
user_id                             ...                                       
35           0.0      0.0      0.0  ...      0.0      0.0      0.0      0.0   
2132         0.0      0.0      0.0  ...      0.0      0.0      0.0      0.0   
2968         0.0      0.0      0.0  ...      0.0      0.0      0.0      0.0   
4170         0.0      0.0      0.0  ...      0.0      0.0      0.0      0.0

##### Matrix Factorization

The matrix factorization function below accepts a matrix R, which is the User x Item matrix or the Rating Matrix. It then creates matrix P and matrix Q using stochasitic gradient descent. Matrix P (User Matrix) is the User x Factors matrix and matrix Q (Item Matrix) is the Item x factor matrix. K refers to the number of features, steps are the number of ephocs per gradient, alpha is the learning rate, and beta is the regularization coefficient. 

eij, or the rating, is calculated with the dot product of P(i) and Q(j). P represents how much the user likes a certain latent factor and Q refers to the how related an item is to a latent factor. 

In [9]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    
    ### R = The user x item rating matrix (m x n)
    ### P = Initial user-factor matrix (m x k)
    ### Q = Initial item-factor matrix (n x k)
    ### K = The number of latent factors (features)
    ### steps = The number of epochs in gradient descent
    ### alpha = The learning rate for gradient descent
    ### beta = The regularization coefficient
    
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        ### update P and Q based on the partial derivatives
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
        print("Step %d of %d; Error: %0.5f; Time: %0.2f" %(step+1, steps, e, time()))
    return P, Q.T

##### P and Q
P and Q are initialized using the dimensions of the Ratings matrix (M x N). Matrix factorization is applied and P and Q are saved.

In [10]:
M = R1.shape[0]
N = R1.shape[1]
K = 12
P = np.random.rand(M,K)
Q = np.random.rand(N,K)

from time import time
t0 = time()

fP, fQ = matrix_factorization(R1, P, Q, 12, steps=3000, alpha=0.0002, beta=0.02)
print("done in %0.3fs." % (time() - t0))


Step 1 of 3000; Error: 34164.91999; Time: 1605824996.99
Step 2 of 3000; Error: 33952.90912; Time: 1605824997.98
Step 3 of 3000; Error: 33741.44503; Time: 1605824998.94
Step 4 of 3000; Error: 33530.53096; Time: 1605825000.18
Step 5 of 3000; Error: 33320.17020; Time: 1605825001.45
Step 6 of 3000; Error: 33110.36606; Time: 1605825002.69
Step 7 of 3000; Error: 32901.12187; Time: 1605825004.08
Step 8 of 3000; Error: 32692.44100; Time: 1605825005.64
Step 9 of 3000; Error: 32484.32683; Time: 1605825007.12
Step 10 of 3000; Error: 32276.78279; Time: 1605825008.50
Step 11 of 3000; Error: 32069.81232; Time: 1605825009.99
Step 12 of 3000; Error: 31863.41886; Time: 1605825011.25
Step 13 of 3000; Error: 31657.60592; Time: 1605825012.79
Step 14 of 3000; Error: 31452.37697; Time: 1605825014.23
Step 15 of 3000; Error: 31247.73555; Time: 1605825015.71
Step 16 of 3000; Error: 31043.68517; Time: 1605825016.93
Step 17 of 3000; Error: 30840.22939; Time: 1605825018.10
Step 18 of 3000; Error: 30637.37175; Tim

Step 145 of 3000; Error: 10762.77214; Time: 1605825172.65
Step 146 of 3000; Error: 10657.02608; Time: 1605825173.63
Step 147 of 3000; Error: 10552.06654; Time: 1605825174.64
Step 148 of 3000; Error: 10447.89158; Time: 1605825175.62
Step 149 of 3000; Error: 10344.49918; Time: 1605825176.57
Step 150 of 3000; Error: 10241.88729; Time: 1605825177.53
Step 151 of 3000; Error: 10140.05380; Time: 1605825178.47
Step 152 of 3000; Error: 10038.99659; Time: 1605825179.40
Step 153 of 3000; Error: 9938.71344; Time: 1605825180.30
Step 154 of 3000; Error: 9839.20212; Time: 1605825181.26
Step 155 of 3000; Error: 9740.46035; Time: 1605825182.21
Step 156 of 3000; Error: 9642.48579; Time: 1605825183.24
Step 157 of 3000; Error: 9545.27608; Time: 1605825184.39
Step 158 of 3000; Error: 9448.82880; Time: 1605825186.02
Step 159 of 3000; Error: 9353.14148; Time: 1605825187.17
Step 160 of 3000; Error: 9258.21164; Time: 1605825188.41
Step 161 of 3000; Error: 9164.03672; Time: 1605825189.68
Step 162 of 3000; Error

Step 289 of 3000; Error: 2147.67847; Time: 1605825336.20
Step 290 of 3000; Error: 2122.10287; Time: 1605825337.52
Step 291 of 3000; Error: 2096.82990; Time: 1605825338.72
Step 292 of 3000; Error: 2071.85653; Time: 1605825339.91
Step 293 of 3000; Error: 2047.17976; Time: 1605825341.00
Step 294 of 3000; Error: 2022.79659; Time: 1605825342.14
Step 295 of 3000; Error: 1998.70406; Time: 1605825343.40
Step 296 of 3000; Error: 1974.89922; Time: 1605825344.52
Step 297 of 3000; Error: 1951.37914; Time: 1605825345.69
Step 298 of 3000; Error: 1928.14092; Time: 1605825346.73
Step 299 of 3000; Error: 1905.18166; Time: 1605825347.79
Step 300 of 3000; Error: 1882.49851; Time: 1605825348.81
Step 301 of 3000; Error: 1860.08860; Time: 1605825349.82
Step 302 of 3000; Error: 1837.94912; Time: 1605825350.82
Step 303 of 3000; Error: 1816.07725; Time: 1605825351.86
Step 304 of 3000; Error: 1794.47020; Time: 1605825353.00
Step 305 of 3000; Error: 1773.12522; Time: 1605825354.12
Step 306 of 3000; Error: 1752.0

Step 435 of 3000; Error: 432.21572; Time: 1605825510.82
Step 436 of 3000; Error: 428.43726; Time: 1605825512.02
Step 437 of 3000; Error: 424.71105; Time: 1605825513.35
Step 438 of 3000; Error: 421.03640; Time: 1605825514.68
Step 439 of 3000; Error: 417.41262; Time: 1605825516.12
Step 440 of 3000; Error: 413.83901; Time: 1605825517.39
Step 441 of 3000; Error: 410.31492; Time: 1605825518.63
Step 442 of 3000; Error: 406.83968; Time: 1605825519.97
Step 443 of 3000; Error: 403.41263; Time: 1605825521.37
Step 444 of 3000; Error: 400.03312; Time: 1605825522.79
Step 445 of 3000; Error: 396.70052; Time: 1605825523.98
Step 446 of 3000; Error: 393.41419; Time: 1605825525.09
Step 447 of 3000; Error: 390.17351; Time: 1605825526.15
Step 448 of 3000; Error: 386.97787; Time: 1605825527.18
Step 449 of 3000; Error: 383.82666; Time: 1605825528.24
Step 450 of 3000; Error: 380.71927; Time: 1605825529.27
Step 451 of 3000; Error: 377.65512; Time: 1605825530.30
Step 452 of 3000; Error: 374.63362; Time: 160582

Step 582 of 3000; Error: 196.30718; Time: 1605825680.69
Step 583 of 3000; Error: 195.83714; Time: 1605825681.85
Step 584 of 3000; Error: 195.37374; Time: 1605825682.89
Step 585 of 3000; Error: 194.91688; Time: 1605825683.91
Step 586 of 3000; Error: 194.46649; Time: 1605825684.92
Step 587 of 3000; Error: 194.02246; Time: 1605825685.93
Step 588 of 3000; Error: 193.58470; Time: 1605825686.95
Step 589 of 3000; Error: 193.15313; Time: 1605825687.99
Step 590 of 3000; Error: 192.72766; Time: 1605825689.04
Step 591 of 3000; Error: 192.30819; Time: 1605825690.07
Step 592 of 3000; Error: 191.89465; Time: 1605825691.08
Step 593 of 3000; Error: 191.48695; Time: 1605825692.13
Step 594 of 3000; Error: 191.08500; Time: 1605825693.18
Step 595 of 3000; Error: 190.68873; Time: 1605825694.20
Step 596 of 3000; Error: 190.29805; Time: 1605825695.25
Step 597 of 3000; Error: 189.91289; Time: 1605825696.29
Step 598 of 3000; Error: 189.53316; Time: 1605825697.43
Step 599 of 3000; Error: 189.15879; Time: 160582

...

Step 2320 of 3000; Error: 162.29486; Time: 1605827666.69
Step 2321 of 3000; Error: 162.29477; Time: 1605827667.98
Step 2322 of 3000; Error: 162.29467; Time: 1605827669.22
Step 2323 of 3000; Error: 162.29457; Time: 1605827670.47
Step 2324 of 3000; Error: 162.29448; Time: 1605827671.68
Step 2325 of 3000; Error: 162.29438; Time: 1605827672.83
Step 2326 of 3000; Error: 162.29428; Time: 1605827674.05
Step 2327 of 3000; Error: 162.29418; Time: 1605827675.26
Step 2328 of 3000; Error: 162.29409; Time: 1605827676.47
Step 2329 of 3000; Error: 162.29399; Time: 1605827677.62
Step 2330 of 3000; Error: 162.29389; Time: 1605827678.83
Step 2331 of 3000; Error: 162.29380; Time: 1605827680.00
Step 2332 of 3000; Error: 162.29370; Time: 1605827681.16
Step 2333 of 3000; Error: 162.29360; Time: 1605827682.39
Step 2334 of 3000; Error: 162.29350; Time: 1605827683.58
Step 2335 of 3000; Error: 162.29341; Time: 1605827684.88
Step 2336 of 3000; Error: 162.29331; Time: 1605827686.13
Step 2337 of 3000; Error: 162.2

Step 2464 of 3000; Error: 162.28090; Time: 1605827830.01
Step 2465 of 3000; Error: 162.28080; Time: 1605827831.29
Step 2466 of 3000; Error: 162.28071; Time: 1605827832.59
Step 2467 of 3000; Error: 162.28061; Time: 1605827833.81
Step 2468 of 3000; Error: 162.28051; Time: 1605827835.10
Step 2469 of 3000; Error: 162.28042; Time: 1605827836.37
Step 2470 of 3000; Error: 162.28032; Time: 1605827837.60
Step 2471 of 3000; Error: 162.28022; Time: 1605827838.90
Step 2472 of 3000; Error: 162.28013; Time: 1605827839.99
Step 2473 of 3000; Error: 162.28003; Time: 1605827841.19
Step 2474 of 3000; Error: 162.27993; Time: 1605827842.93
Step 2475 of 3000; Error: 162.27984; Time: 1605827844.53
Step 2476 of 3000; Error: 162.27974; Time: 1605827845.70
Step 2477 of 3000; Error: 162.27964; Time: 1605827847.16
Step 2478 of 3000; Error: 162.27955; Time: 1605827848.38
Step 2479 of 3000; Error: 162.27945; Time: 1605827849.51
Step 2480 of 3000; Error: 162.27935; Time: 1605827850.86
Step 2481 of 3000; Error: 162.2

Step 2608 of 3000; Error: 162.26697; Time: 1605828043.73
Step 2609 of 3000; Error: 162.26688; Time: 1605828047.41
Step 2610 of 3000; Error: 162.26678; Time: 1605828050.30
Step 2611 of 3000; Error: 162.26668; Time: 1605828053.81
Step 2612 of 3000; Error: 162.26659; Time: 1605828056.71
Step 2613 of 3000; Error: 162.26649; Time: 1605828059.13
Step 2614 of 3000; Error: 162.26639; Time: 1605828061.06
Step 2615 of 3000; Error: 162.26630; Time: 1605828063.33
Step 2616 of 3000; Error: 162.26620; Time: 1605828065.41
Step 2617 of 3000; Error: 162.26610; Time: 1605828067.58
Step 2618 of 3000; Error: 162.26601; Time: 1605828069.24
Step 2619 of 3000; Error: 162.26591; Time: 1605828070.85
Step 2620 of 3000; Error: 162.26581; Time: 1605828072.35
Step 2621 of 3000; Error: 162.26572; Time: 1605828073.93
Step 2622 of 3000; Error: 162.26562; Time: 1605828075.43
Step 2623 of 3000; Error: 162.26552; Time: 1605828077.03
Step 2624 of 3000; Error: 162.26543; Time: 1605828078.59
Step 2625 of 3000; Error: 162.2

Step 2752 of 3000; Error: 162.25308; Time: 1605828264.21
Step 2753 of 3000; Error: 162.25298; Time: 1605828265.54
Step 2754 of 3000; Error: 162.25288; Time: 1605828266.84
Step 2755 of 3000; Error: 162.25279; Time: 1605828268.27
Step 2756 of 3000; Error: 162.25269; Time: 1605828269.74
Step 2757 of 3000; Error: 162.25259; Time: 1605828271.26
Step 2758 of 3000; Error: 162.25250; Time: 1605828272.82
Step 2759 of 3000; Error: 162.25240; Time: 1605828274.45
Step 2760 of 3000; Error: 162.25231; Time: 1605828276.02
Step 2761 of 3000; Error: 162.25221; Time: 1605828277.80
Step 2762 of 3000; Error: 162.25211; Time: 1605828279.99
Step 2763 of 3000; Error: 162.25202; Time: 1605828282.15
Step 2764 of 3000; Error: 162.25192; Time: 1605828283.73
Step 2765 of 3000; Error: 162.25182; Time: 1605828285.53
Step 2766 of 3000; Error: 162.25173; Time: 1605828287.56
Step 2767 of 3000; Error: 162.25163; Time: 1605828289.25
Step 2768 of 3000; Error: 162.25153; Time: 1605828291.34
Step 2769 of 3000; Error: 162.2

Step 2896 of 3000; Error: 162.23921; Time: 1605828493.39
Step 2897 of 3000; Error: 162.23912; Time: 1605828494.78
Step 2898 of 3000; Error: 162.23902; Time: 1605828496.20
Step 2899 of 3000; Error: 162.23892; Time: 1605828497.62
Step 2900 of 3000; Error: 162.23883; Time: 1605828499.27
Step 2901 of 3000; Error: 162.23873; Time: 1605828500.71
Step 2902 of 3000; Error: 162.23863; Time: 1605828502.22
Step 2903 of 3000; Error: 162.23854; Time: 1605828503.56
Step 2904 of 3000; Error: 162.23844; Time: 1605828504.83
Step 2905 of 3000; Error: 162.23835; Time: 1605828506.09
Step 2906 of 3000; Error: 162.23825; Time: 1605828507.33
Step 2907 of 3000; Error: 162.23815; Time: 1605828508.74
Step 2908 of 3000; Error: 162.23806; Time: 1605828510.29
Step 2909 of 3000; Error: 162.23796; Time: 1605828511.69
Step 2910 of 3000; Error: 162.23787; Time: 1605828513.05
Step 2911 of 3000; Error: 162.23777; Time: 1605828514.63
Step 2912 of 3000; Error: 162.23767; Time: 1605828516.06
Step 2913 of 3000; Error: 162.2

In [ ]:
outP = open("rr_p.csv", "w")
outQ = open("rr_q.csv", "w")
np.savetxt(outP, fP, delimiter=',', fmt='%1.4f')
np.savetxt(outQ, fQ, delimiter=',', fmt='%1.4f')

##### Predicted Ratings
This computes the predicted rating for each user and item. 

In [ ]:
Preds = np.dot(fP,fQ.T)

In [ ]:
outPreds = open("runway_preds.csv", "w")
np.savetxt(outPreds, Preds, delimiter=',', fmt='%1.4f')

##### Mean Asbolute Error
Lastly we calculate the Mean Absolute Error (MAE) for each user as well as the overall MAE for the disctict users and items. 

In [11]:
totCount = 0
totError = 0

for u in range(M):
    err_u = 0
    rateCount_u = 0
    for j in range(N):
        if (R1[u,j] > 0): ### Only use known ratings computing error
            rateCount_u += 1
            err_u += abs(np.dot(fP[u],fQ[j]) - R1[u,j])
    print("Mean Absolute Error for User %d = %0.3f" %(u, err_u/rateCount_u))
    totCount += rateCount_u
    totError += err_u
print
print("Overall Mean Absolute Error = %0.3f" %(totError/totCount))

Mean Absolute Error for User 0 = 0.010
Mean Absolute Error for User 1 = 0.010
Mean Absolute Error for User 2 = 0.010
Mean Absolute Error for User 3 = 0.009
Mean Absolute Error for User 4 = 0.010
Mean Absolute Error for User 5 = 0.010
Mean Absolute Error for User 6 = 0.010
Mean Absolute Error for User 7 = 0.010
Mean Absolute Error for User 8 = 0.010
Mean Absolute Error for User 9 = 0.010
Mean Absolute Error for User 10 = 0.009
Mean Absolute Error for User 11 = 0.010
Mean Absolute Error for User 12 = 0.010
Mean Absolute Error for User 13 = 0.010
Mean Absolute Error for User 14 = 0.010
Mean Absolute Error for User 15 = 0.009
Mean Absolute Error for User 16 = 0.010
Mean Absolute Error for User 17 = 0.010
Mean Absolute Error for User 18 = 0.010
Mean Absolute Error for User 19 = 0.010
Mean Absolute Error for User 20 = 0.009
Mean Absolute Error for User 21 = 0.010
Mean Absolute Error for User 22 = 0.010
Mean Absolute Error for User 23 = 0.010
Mean Absolute Error for User 24 = 0.009
Mean Absol

Mean Absolute Error for User 252 = 0.009
Mean Absolute Error for User 253 = 0.010
Mean Absolute Error for User 254 = 0.010
Mean Absolute Error for User 255 = 0.009
Mean Absolute Error for User 256 = 0.010
Mean Absolute Error for User 257 = 0.010
Mean Absolute Error for User 258 = 0.010
Mean Absolute Error for User 259 = 0.010
Mean Absolute Error for User 260 = 0.010
Mean Absolute Error for User 261 = 0.010
Mean Absolute Error for User 262 = 0.010
Mean Absolute Error for User 263 = 0.010
Mean Absolute Error for User 264 = 0.010
Mean Absolute Error for User 265 = 0.010
Mean Absolute Error for User 266 = 0.010
Mean Absolute Error for User 267 = 0.010
Mean Absolute Error for User 268 = 0.010
Mean Absolute Error for User 269 = 0.010
Mean Absolute Error for User 270 = 0.010
Mean Absolute Error for User 271 = 0.010
Mean Absolute Error for User 272 = 0.010
Mean Absolute Error for User 273 = 0.010
Mean Absolute Error for User 274 = 0.010
Mean Absolute Error for User 275 = 0.010
Mean Absolute Er

Mean Absolute Error for User 533 = 0.010
Mean Absolute Error for User 534 = 0.010
Mean Absolute Error for User 535 = 0.010
Mean Absolute Error for User 536 = 0.009
Mean Absolute Error for User 537 = 0.010
Mean Absolute Error for User 538 = 0.009
Mean Absolute Error for User 539 = 0.010
Mean Absolute Error for User 540 = 0.010
Mean Absolute Error for User 541 = 0.010
Mean Absolute Error for User 542 = 0.010
Mean Absolute Error for User 543 = 0.009
Mean Absolute Error for User 544 = 0.010
Mean Absolute Error for User 545 = 0.010
Mean Absolute Error for User 546 = 0.010
Mean Absolute Error for User 547 = 0.009
Mean Absolute Error for User 548 = 0.010
Mean Absolute Error for User 549 = 0.010
Mean Absolute Error for User 550 = 0.010
Mean Absolute Error for User 551 = 0.010
Mean Absolute Error for User 552 = 0.010
Mean Absolute Error for User 553 = 0.010
Mean Absolute Error for User 554 = 0.010
Mean Absolute Error for User 555 = 0.010
Mean Absolute Error for User 556 = 0.009
Mean Absolute Er

Mean Absolute Error for User 798 = 0.010
Mean Absolute Error for User 799 = 0.010
Mean Absolute Error for User 800 = 0.010
Mean Absolute Error for User 801 = 0.010
Mean Absolute Error for User 802 = 0.010
Mean Absolute Error for User 803 = 0.010
Mean Absolute Error for User 804 = 0.010
Mean Absolute Error for User 805 = 0.010
Mean Absolute Error for User 806 = 0.010
Mean Absolute Error for User 807 = 0.010
Mean Absolute Error for User 808 = 0.010
Mean Absolute Error for User 809 = 0.007
Mean Absolute Error for User 810 = 0.010
Mean Absolute Error for User 811 = 0.010
Mean Absolute Error for User 812 = 0.010
Mean Absolute Error for User 813 = 0.009
Mean Absolute Error for User 814 = 0.009
Mean Absolute Error for User 815 = 0.010
Mean Absolute Error for User 816 = 0.010
Mean Absolute Error for User 817 = 0.010
Mean Absolute Error for User 818 = 0.010
Mean Absolute Error for User 819 = 0.010
Mean Absolute Error for User 820 = 0.009
Mean Absolute Error for User 821 = 0.010
Mean Absolute Er

The Overall MAE using distinct users (~900) was .010 while the overall MAE for 2500 users was .018. The MAE is pretty small and indicates that the matrix factorization performed well using stochastic gradient descent. 